In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
#파일 직접 업로드하기
from google.colab import files
myfile = files.upload()

Saving labeled_lyrics_cleaned.csv to labeled_lyrics_cleaned.csv


In [3]:
#io와 pandas 모듈 import
import io
import pandas as pd

In [4]:
#pd.read_csv로 csv파일 불러오기
data = pd.read_csv(io.BytesIO(myfile['labeled_lyrics_cleaned.csv']))

data.drop('Unnamed: 0', axis=1, inplace=True) # 인덱스 열 제거
data.rename(columns={'seq':'lyrics'}, inplace= True) # 가사 열 이름 변경

data['lyrics'] = data['lyrics'].str.lower() #가사 소문자로변경
data['lyrics'] = data['lyrics'].str.replace(',','') # 가사 콤마 제거

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
#표제어(Lemma) 추출 - 기본 사전형 단어를 추출
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [6]:
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

In [7]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
data['lem'] = data['lyrics'].apply(lemmatize_words)

In [9]:
data['lem'] = data['lem'].str.replace(pat=r'\(.*\)|\s-\s.*', repl=r' ', regex=True) #괄호() 가사 제거
data['lem'] = data['lem'].str.replace(pat=r"'", repl=r' ', regex=True) 
# Trippin' off that Grigio, mobbin', lights low 이런 가사들 때문에 '(따옴표) 제거

In [10]:
# 가사 토큰화 

# from nltk.tokenize import word_tokenize
# data['cleaned_lyrics'] = data['lem'].apply(word_tokenize)
# [no, no, i, ai, n't, ever, trap, out, the, ban... 이상하게 나뉨

data['cleaned_lyrics'] = data['lem'].str.split() #띄어쓰기로 나누기

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
from nltk.corpus import stopwords
stop = stopwords.words('english') #불용어 사전 가져오기

In [13]:
# 불용어 단어 더 추가
stop.extend(["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "pa","page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz", "'"])

In [14]:
# Remove all stopwords from the lyrics 
data['cleaned_lyrics'] = data['cleaned_lyrics'].apply(lambda x: [item for item in x if item not in stop])

In [15]:
# !pip install -U LeXmo
# from LeXmo import LeXmo

In [16]:
# data['lyrics_emo']=data['lem'].apply(lambda x : LeXmo.LeXmo(x))

In [17]:
# data.to_csv('spotify_lyrics.csv', index=False)

In [18]:
# data['test']=data['lem'].apply(lambda x : x)

In [19]:
# data.drop('test', axis=1, inplace=True)

In [20]:
# data

In [21]:
pip install NRCLex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 396 kB 32.5 MB/s 
  Created wheel for NRCLex: filename=NRCLex-3.0.0-py3-none-any.whl size=43329 sha256=a61ae716f60db81b82e3ca292173d4004ef67d220b6585060b77c2e36deb6681
  Stored in directory: /root/.cache/pip/wheels/83/95/c0/42b43fb15eb48e4f5a67cba8915540cb2783591c59c037a9e5
Successfully built NRCLex


In [22]:

# Import required modules
from nrclex import NRCLex

In [23]:
# # Assign list of strings
# text = ['hate', 'lovely', 'person', 'worst']
 
# # Iterate through list
# for i in range(len(text)):
 
#     # Create object
#     emotion = NRCLex(text[i])
 
#     # Classify emotion
#     print('\n\n', text[i], ': ', emotion.top_emotions)

In [24]:
# from nrclex import NRCLex
# text = 'hate'
# emotion = NRCLex(text)
# print(emotion.raw_emotion_scores) 

In [25]:
# NRCLex(data['lem'][0]).raw_emotion_scores

In [28]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
data['lyrics_emo']=data['lem'].apply(lambda x : NRCLex(x).raw_emotion_scores)

In [30]:
data

,artist,lyrics,song,label,lem,cleaned_lyrics,lyrics_emo
0,Elijah Blake,no no\r\ni ain't ever trapped out the bando\r\...,Everyday,0.626,no no i ain t ever trap out the bando but oh l...,"[trap, bando, lord, wrong, couple, niggas, pla...","{'disgust': 6, 'negative': 11, 'positive': 11,..."
1,Elijah Blake,the drinks go down and smoke goes up i feel my...,Live Till We Die,0.630,the drink go down and smoke go up i feel mysel...,"[drink, smoke, feel, care, lose, crowd, yo, sl...","{'anger': 1, 'disgust': 1, 'fear': 27, 'negati..."
2,Elijah Blake,she don't live on planet earth no more\r\nshe ...,The Otherside,0.240,she don t live on planet earth no more she fin...,"[live, planet, earth, love, venus, word, space...","{'positive': 5, 'trust': 3, 'joy': 1, 'anticip..."
3,Elijah Blake,trippin' off that grigio mobbin' lights low\r\...,Pinot,0.536,trippin off that grigio mobbin light low tri...,"[trippin, grigio, mobbin, light, low, trippin,...","{'joy': 16, 'positive': 23, 'fear': 6, 'surpri..."
4,Elijah Blake,i see a midnight panther so gallant and so bra...,Shadows & Diamonds,0.371,i see a midnight panther so gallant and so bra...,"[midnight, panther, gallant, brave, answer, th...","{'positive': 14, 'joy': 12, 'trust': 11, 'nega..."
...,...,...,...,...,...,...,...
158348,Adam Green,and we live on borrowed time\r\nbut this heads...,Friends of Mine,0.737,and we live on borrow time but this headshot s...,"[live, borrow, time, headshot, pretty, good, s...","{'anticipation': 7, 'joy': 11, 'positive': 11,..."
158349,Adam Green,frozin in time forever\r\ncarrying that torch ...,Frozen in Time,0.482,frozin in time forever carry that torch for so...,"[frozin, time, forever, carry, torch, long, he...","{'anticipation': 12, 'fear': 2, 'trust': 6, 'j..."
158350,Adam Green,hard to be a girl. \r\nso nice to be a boy. \r...,Hard to Be a Girl,0.733,hard to be a girl. so nice to be a boy. in my ...,"[hard, girl., nice, boy., room, night., pretty...","{'disgust': 5, 'negative': 5, 'anticipation': ..."
158351,Adam Green,i want to chose to die\r\nand be buried with a...,I Wanna Die,0.361,i want to choose to die and be bury with a rub...,"[choose, die, bury, rubix, cube, sleep, inside...","{'fear': 5, 'negative': 8, 'sadness': 7, 'trus..."


In [32]:
data.to_csv('spotify_lyrics.csv', index=False)